<a href="https://colab.research.google.com/github/nanochiplab/RBFTraffic/blob/main/OdevTraffic_RBF_Gamma0_1_Sgd_Hiper_Gamma0_1_Sgd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image

import os
from keras.utils import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout,Activation,InputLayer,Layer

import seaborn as sns
from sklearn.metrics import confusion_matrix


In [3]:
#!git clone https://github.com/sonu275981/GTSRB---German-Traffic-Sign-Recognition
#cur_path= "GTSRB---German-Traffic-Sign-Recognition/gtsrb-german-traffic-sign/Train/"
!git clone https://github.com/MohamedNennouche/German-Traffic-Sign-Recognition
cur_path= "German-Traffic-Sign-Recognition/Train/"


Cloning into 'German-Traffic-Sign-Recognition'...
remote: Enumerating objects: 52027, done.
remote: Total 52027 (delta 0), reused 0 (delta 0), pack-reused 52027 (from 1)
Receiving objects: 100% (52027/52027), 300.65 MiB | 33.19 MiB/s, done.
Resolving deltas: 100% (44/44), done.
Updating files: 100% (101736/101736), done.


In [4]:
#from google.colab import drive
#drive.mount('/content/drive')

#cur_path = "/content/drive/MyDrive/Mlp/Train"


data = []
labels = []
classes = 43  # Toplam sınıf sayısı

# Veri listeleri
X = []
y = []

print("")
print("Görüntüler Yükleniyor...")
print("************************........:")
print("")

# Görüntüleri yükleme

# Görüntüleri yükleme
for i in range(classes):
    path = os.path.join(cur_path, str(i))


    # Özyinelemeli olarak tüm dosyaları gez
    for root, dirs, files in os.walk(path):
        for file in files:
            try:
                image_path = os.path.join(root, file)

                # Görüntü dosyasını yükle ve işle
                image = Image.open(image_path)
                image = image.resize((30, 30))
                image = np.array(image)

                # Veri ve etiketleri ekle
                #data.append([image, i])
                data.append(image)
                labels.append(i)  # Etiketleri ayrı olarak ekleyin
            except Exception as e:
                print(f"Hata: Görüntü yüklenemedi. Dosya: {image_path}, Hata: {e}")

print(f"Toplam yüklenen görüntü sayısı: {len(data)}")
print("************************........:")

#x = []
#y = []
"""
X = []
y = []

for features,label in data:
    X.append(features)
    y.append(label)
"""
#Converting lists into numpy arrays
X = np.array(data)
y = np.array(labels)


Görüntüler Yükleniyor...
************************........:

Toplam yüklenen görüntü sayısı: 39209
************************........:


In [5]:
!pip install scikeras

In [6]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Layer, Flatten
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Özel RBF katmanı tanımı
class RBFLayer(Layer):
    def __init__(self, num_units, gamma, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.num_units = num_units
        self.gamma = gamma

    def build(self, input_shape):
        self.centers = self.add_weight(name='centers',
                                       shape=(self.num_units, input_shape[-1]),
                                       initializer='random_normal',
                                       trainable=True)
        self.built = True

    def call(self, inputs):
        # RBF fonksiyonu: exp(-gamma * ||inputs - centers||^2)
        diff = tf.expand_dims(inputs, axis=1) - self.centers
        sq_dist = tf.reduce_sum(tf.square(diff), axis=-1)
        return tf.exp(-self.gamma * sq_dist)

# Veriyi normalize etme ve sınıflara ayırma
X = X / 255.0  # Piksel değerlerini 0-1 arasında normalize et
y = to_categorical(labels, num_classes=len(np.unique(labels)))  # Etiketleri one-hot encode et

# Eğitim ve test setlerini oluştur
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Modeli oluşturma
model = Sequential([
    Input(shape=(30, 30, 3)),   # Görüntü boyutları (30x30x3)
    Flatten(),                  # Görüntüyü düzleştir
    RBFLayer(num_units=100, gamma=0.1),  # RBF katmanı
    Dense(len(y[0]), activation='softmax')  # Çıktı katmanı (sınıf sayısı kadar)
])

# Modeli derleme
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli eğitme
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

# Test doğruluğunu değerlendirme
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test doğruluğu: {test_acc:.2f}")



Epoch 1/10
429/429 ━━━━━━━━━━━━━━━━━━━━ 105s 242ms/step - accuracy: 0.0517 - loss: 3.7482 - val_accuracy: 0.0630 - val_loss: 3.7095
Epoch 2/10
429/429 ━━━━━━━━━━━━━━━━━━━━ 131s 218ms/step - accuracy: 0.0632 - loss: 3.6984 - val_accuracy: 0.0684 - val_loss: 3.6660
Epoch 3/10
429/429 ━━━━━━━━━━━━━━━━━━━━ 143s 220ms/step - accuracy: 0.0657 - loss: 3.6586 - val_accuracy: 0.0712 - val_loss: 3.6307
Epoch 4/10
429/429 ━━━━━━━━━━━━━━━━━━━━ 157s 255ms/step - accuracy: 0.0698 - loss: 3.6213 - val_accuracy: 0.0706 - val_loss: 3.6024
Epoch 5/10
429/429 ━━━━━━━━━━━━━━━━━━━━ 100s 234ms/step - accuracy: 0.0672 - loss: 3.5960 - val_accuracy: 0.0705 - val_loss: 3.5795
Epoch 6/10
429/429 ━━━━━━━━━━━━━━━━━━━━ 98s 228ms/step - accuracy: 0.0697 - loss: 3.5743 - val_accuracy: 0.0705 - val_loss: 3.5611
Epoch 7/10
429/429 ━━━━━━━━━━━━━━━━━━━━ 101s 236ms/step - accuracy: 0.0709 - loss: 3.5571 - val_accuracy: 0.0706 - val_loss: 3.5462
Epoch 8/10
429/429 ━━━━━━━━━━━━━━━━━━━━ 102s 237ms/step - accuracy: 0.0670 - 

In [8]:
def create_model(num_units=100, gamma=0.1, optimizer='adam'):
    model = Sequential([
        Input(shape=(30, 30, 3)),   # Görüntü boyutları (30x30x3)
        Flatten(),                  # Görüntüyü düzleştir
        RBFLayer(num_units=num_units, gamma=gamma),  # RBF katmanı
        Dense(len(y_train[0]), activation='softmax')  # Çıktı katmanı (sınıf sayısı kadar)
    ])
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

#Modeli bir KerasClassifier içine sarmalayarak, Scikit-learn’ün arama metodlarıyla uyumlu hale getiriyoruz.
keras_model = KerasClassifier(model=create_model, verbose=0)

#Hiperparametre Alanı ve RandomizedSearchCV
#Hiperparametreler için arama alanını belirliyoruz. Burada num_units, gamma, optimizer, batch_size, ve epochs
#gibi parametrelerin aralığını belirtebiliriz.

# Hiperparametre arama alanı
# Hiperparametre arama alanı
param_distributions = {
    'model__num_units': [50],            # RBF katmanı birim sayısı
    'model__gamma': [0.1],           # RBF gamma parametresi
    'model__optimizer': ['sgd'], # Farklı optimizasyon algoritmaları
    'batch_size': [64],            # Batch size
    'epochs': [10, 10]
}
"""
param_distributions = {
    'model__num_units': [50, 100, 150],            # RBF katmanı birim sayısı
    'model__gamma': [0.01, 0.1, 1, 10],           # RBF gamma parametresi
    'model__optimizer': ['adam', 'sgd', 'rmsprop'], # Farklı optimizasyon algoritmaları
    'batch_size': [32, 64, 128],            # Batch size
    'epochs': [10, 20]                      # Eğitim döngü sayısı
}
"""
#******************************************
# RandomizedSearchCV yapılandırma
random_search = RandomizedSearchCV(
    estimator=keras_model,
    param_distributions=param_distributions,
    n_iter=5,  # Arama yapılacak kombinasyon sayısı
    cv=3,       # 3 katlı çapraz doğrulama
    verbose=2,
    n_jobs=-1   # Paralel çalıştırma (tüm çekirdekler)
)
#Modeli Eğitme ve En İyi Sonuçları Bulma
#Arama işlemini başlatıyoruz ve en iyi parametre kombinasyonunu buluyoruz.
random_search.fit(X_train, y_train)

# En iyi parametreleri ve doğruluğu görüntüleme
print("En iyi parametreler:", random_search.best_params_)
print("En iyi doğruluk:", random_search.best_score_)

#Seçilen Parametrelerle Modeli Test Etme
#En iyi parametrelerle oluşturulan modeli test etmek için:

best_model = random_search.best_estimator_
test_loss, test_acc = best_model.model_.evaluate(X_test, y_test)
print(f"Test doğruluğu (optimized): {test_acc:.2f}")


Fitting 3 folds for each of 2 candidates, totalling 6 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 2 is smaller than n_iter=5. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


En iyi parametreler: {'model__optimizer': 'sgd', 'model__num_units': 50, 'model__gamma': 0.1, 'epochs': 10, 'batch_size': 64}
En iyi doğruluk: 0.06594778147697429
368/368 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.0619 - loss: 3.5187
Test doğruluğu (optimized): 0.06


In [ ]:
# Parameter grid for GridSearchCV
param_grid = {
    'model__num_units': [50, 100, 150],  # Updated: model__num_units
    'model__gamma': [0.1, 0.5, 1.0],  # Updated: model__gamma
}

# Create GridSearchCV object
grid_search = GridSearchCV(
    estimator=KerasClassifier(model=create_model), # Updated line
    param_grid=param_grid,
    cv=3,  # Number of cross-validation folds
    verbose=2,
)

# Fit the model
grid_search.fit(X_train, y_train)

# Get the best parameters and score
print("Best parameters found by GridSearchCV:", grid_search.best_params_)
print("Best score found by GridSearchCV:", grid_search.best_score_)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
572/572 ━━━━━━━━━━━━━━━━━━━━ 16s 27ms/step - accuracy: 0.0704 - loss: 3.6683
286/286 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step
[CV] END ..............model__gamma=0.1, model__num_units=50; total time=  20.4s
572/572 ━━━━━━━━━━━━━━━━━━━━ 16s 27ms/step - accuracy: 0.0776 - loss: 3.6585
286/286 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step
[CV] END ..............model__gamma=0.1, model__num_units=50; total time=  25.2s
572/572 ━━━━━━━━━━━━━━━━━━━━ 19s 32ms/step - accuracy: 0.0716 - loss: 3.6603
286/286 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step
[CV] END ..............model__gamma=0.1, model__num_units=50; total time=  23.5s
572/572 ━━━━━━━━━━━━━━━━━━━━ 60s 103ms/step - accuracy: 0.0710 - loss: 3.6588
286/286 ━━━━━━━━━━━━━━━━━━━━ 11s 37ms/step
[CV] END .............model__gamma=0.1, model__num_units=100; total time= 1.6min
572/572 ━━━━━━━━━━━━━━━━━━━━ 61s 106ms/step - accuracy: 0.0715 - loss: 3.6559
286/286 ━━━━━━━━━━━━━━━━━━━━ 9s 30ms/step
[CV] END ...........